In [6]:
import os
import torch

# Import our module containing helper functions
import gravann.polyhedral
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

# plotting stuff
%matplotlib notebook

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ", device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Will use device  cpu


We first chose one of the following samples:

    "eros", "bennu", "itokawa", "churyumov-gerasimenko", "planetesimal", "torus", "bennu_nu", "itokawa_nu", "planetesimal_nu"

In [7]:
sample_name = "churyumov-gerasimenko"
print("Name of the chosen body: ", sample_name)

Name of the chosen body:  churyumov-gerasimenko


Next, we can plot a comparison between the polyhedral gravity model's labels and the mascon labels:

In [9]:
gravann.polyhedral.plot_polyhedral_mascon_acceleration(sample_name,
                                                       plane="XY",
                                                       N=1000,
                                                       logscale=False,
                                                       altitude=0.001)

Sampling points at altitude
Discarding 0 of 18294 points in altitude sampler which did not meet requested altitude.


KeyboardInterrupt: 